In [1]:
%load_ext twisted_ipython
%autoawait twisted

In [6]:
from twisted.internet import reactor
import functools

import attr
from txdbus import client

from pyxsession.dbus.marshmallow.schema import from_attrs, DBUS_SCHEMA
from pyxsession.dbus.marshmallow.fields import (
    DBusField, Int32, UInt32, Str, List, Tuple
)
from pyxsession.dbus.signature import signature

      
@attr.s
class Notification:
    appname = attr.ib(default='', metadata={DBUS_SCHEMA: Str()})
    replaces = attr.ib(default=0, metadata={DBUS_SCHEMA: UInt32()})
    icon = attr.ib(default='', metadata={DBUS_SCHEMA: Str()})
    summary = attr.ib(default='', metadata={DBUS_SCHEMA: Str()})
    message = attr.ib(default='', metadata={DBUS_SCHEMA: Str()})
    actions = attr.ib(default=[], metadata={DBUS_SCHEMA: DBusField('as')})
    hints = attr.ib(default={}, metadata={DBUS_SCHEMA: DBusField('a{sv}')})
    timeout = attr.ib(default=1000, metadata={DBUS_SCHEMA: Int32()})

NotificationSchema = from_attrs(Notification)
                                          
schema = NotificationSchema()

In [7]:
notification = Notification(
    appname='jupyter',
    summary='an important message',
    message='my dog',
    timeout=5000,
    hints={'key': 'bro'}
)

In [8]:
dump = schema.dump(notification)

dump

['jupyter', 0, '', 'an important message', 'my dog', [], {'key': 'bro'}, 5000]

In [9]:
load = schema.load(dump)

load

Notification(appname='jupyter', replaces=0, icon='', summary='an important message', message='my dog', actions=[], hints={'key': 'bro'}, timeout=5000)

In [10]:
signature(schema)

'susssasa{sv}i'

In [ ]:
conn = await client.connect(reactor, 'session')

notifier = await conn.getRemoteObject('org.freedesktop.Notifications', '/org/freedesktop/Notifications')

In [ ]:
from txdbus.interface import DBusInterface, Method

iface = DBusInterface(
    'org.freedesktop.Notifications',
    Method('Notify', arguments=signature(schema), returns='u')
)

unreflected = await conn.getRemoteObject(
    'org.freedesktop.Notifications',
    '/org/freedesktop/Notifications',
    iface
)

await unreflected.callRemote('Notify', *schema.dump(notification))